In [7]:
pip install seaborn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install scikit-learn pandas numpy nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import string
import re


In [24]:
# Load the dataset (adjust the file path if necessary)
df = pd.read_csv('email_spam.csv')

# Check the first few rows and columns
print(df.head())
print(f"Dataset columns: {df.columns}")


                                               title  \
0                          ?? the secrets to SUCCESS   
1                    ?? You Earned 500 GCLoot Points   
2                         ?? Your GitHub launch code   
3  [The Virtual Reward Center] Re: ** Clarifications   
4  10-1 MLB Expert Inside, Plus Everything You Ne...   

                                                text      type  
0  Hi James,\n\nHave you claim your complimentary...      spam  
1  \nalt_text\nCongratulations, you just earned\n...  not spam  
2  Here's your GitHub launch code, @Mortyj420!\n ...  not spam  
3  Hello,\n \nThank you for contacting the Virtua...  not spam  
4  Hey Prachanda Rawal,\n\nToday's newsletter is ...      spam  
Dataset columns: Index(['title', 'text', 'type'], dtype='object')


In [25]:
# Function to clean the text data
def clean_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and non-alphanumeric characters (except spaces)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply the clean_text function to the text column
df['cleaned_text'] = df['text'].apply(clean_text)

# Check the cleaned text
print(df[['text', 'cleaned_text']].head())


                                                text  \
0  Hi James,\n\nHave you claim your complimentary...   
1  \nalt_text\nCongratulations, you just earned\n...   
2  Here's your GitHub launch code, @Mortyj420!\n ...   
3  Hello,\n \nThank you for contacting the Virtua...   
4  Hey Prachanda Rawal,\n\nToday's newsletter is ...   

                                        cleaned_text  
0  hi james have you claim your complimentary gif...  
1  alt_text congratulations you just earned 500 y...  
2  heres your github launch code mortyj420 an oct...  
3  hello thank you for contacting the virtual rew...  
4  hey prachanda rawal todays newsletter is jampa...  


In [33]:
# Features: Use the 'cleaned_text' column
X = df['cleaned_text']

# Labels: The 'type' column represents spam (1) or ham (0)
y = df['type']

# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the size of the training and testing sets
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")


Training set size: 67, Test set size: 17


In [34]:
# Initialize the TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')  # Remove English stopwords

# Fit and transform the training data, then transform the test data
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Check the shape of the transformed data (it should be [n_samples, n_features])
print(f"TF-IDF training data shape: {X_train_tfidf.shape}")
print(f"TF-IDF test data shape: {X_test_tfidf.shape}")


TF-IDF training data shape: (67, 1852)
TF-IDF test data shape: (17, 1852)


In [36]:
# Initialize the Naive Bayes classifier
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [31]:
print(df['type'].value_counts())


type
not spam    58
spam        26
Name: count, dtype: int64


In [37]:
# Make predictions on the test set
y_pred = nb_model.predict(X_test_tfidf)

# Evaluate the model: accuracy and classification report
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))  # Set zero_division=1 to handle undefined precision


Accuracy: 0.6470588235294118
Classification Report:
              precision    recall  f1-score   support

    not spam       0.65      1.00      0.79        11
        spam       1.00      0.00      0.00         6

    accuracy                           0.65        17
   macro avg       0.82      0.50      0.39        17
weighted avg       0.77      0.65      0.51        17



In [38]:
# Function to classify a new message as 'Spam' or 'Ham'
def classify_message(message):
    # Clean the input message
    message_cleaned = clean_text(message)
    
    # Convert the cleaned message into a TF-IDF feature vector
    message_tfidf = tfidf.transform([message_cleaned])
    
    # Make the prediction (1 = spam, 0 = ham)
    prediction = nb_model.predict(message_tfidf)
    
    # Return the result as 'Spam' or 'Ham'
    return 'Spam' if prediction[0] == 1 else 'Ham'

# Example usage of the classify_message function
new_message = "Congratulations! You've won a $1000 gift card!"
print(f"The message is: {classify_message(new_message)}")


The message is: Ham
